# Picture to Plate

In [ ]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor, BitsAndBytesConfig
import torch
from qwen_vl_utils import process_vision_info
import nltk
from rouge_score import rouge_scorer

In [152]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Set False for 8-bit
    bnb_4bit_compute_dtype=torch.float16
)

In [153]:
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct", torch_dtype="auto", device_map="auto", quantization_config=bnb_config,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [188]:
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")

#### **Setup**

In "filename" enter the filename from "Test_Images" folder. In "dish" enter the title of the dish (noisy title)

In [281]:
filename = "RedPasta"
image_path = f"./Test_Images/{filename}.jpeg"
dish = "Italian"

## Description Generation

In [282]:
description_prompt = f"""Describe the food image with ingredients, texture etc present.
Try to predict what's the actual name of the dish. 
Dish: {dish}"""

In [283]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": image_path,
            },
            {"type": "text", "text": description_prompt},
        ],
    }
]

In [284]:
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

In [285]:
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
description_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
description_text = description_text[0]
print(description_text)

The dish in the picture is penne pasta served in a creamy tomato sauce. The pasta appears to be coated in a rich, orange-red sauce that has a smooth texture. Garnishing the dish includes finely chopped parsley and grated Parmesan cheese sprinkled on top.

Based on the ingredients and presentation, this dish can be identified as "Penne Arrabbiata," which translates to "Red Sauce Penne" in English. It is a popular Italian pasta dish known for its vibrant red sauce made from tomatoes, garlic, and sometimes red pepper flakes or chili peppers. The dish is typically served with fresh parsley and grated Parmesan cheese as


## Recipe Generation

**Prompt Structure**

In [286]:
recipe_generator = f"""You are a professional recipe generator. Based on the "description" of the food generate a recipe.
Description of the food : {description_text}

Format: The recipe should be in "3-steps". 
Dish: Dish Name
Summary: 
1. <Step 1>
2. <Step 2>
3. <Step 3>

Examples 1: 
Dish: Mustard Fish
Summary: 1. Clean and slice the fish. Soak mustard seeds for 2 hours, then grind with salt and green chillies to a paste. Mix in grated coconut, yoghurt, sugar, turmeric, and mustard oil to make a marinade.
2. Coat the fish pieces with the mixture, place them in a tiffin box with halved green chillies on top, fasten the lid, and marinate for at least 15 minutes.
3. Steam the sealed tiffin in a covered pan with hot water for 15 minutes. Let it rest for 5 minutes before serving.

Example 2:
Dish: Butter Chicken
Summary: 1. Marinate boneless chicken pieces in yogurt, lemon juice, ginger-garlic paste, chili powder, and garam masala for at least 1 hour. Then grill, bake, or pan-fry until cooked and slightly charred.
2. In a pan, heat butter and sauté ginger, garlic, and tomatoes until soft. Blend this into a smooth puree, then return to the pan and add cream, kasuri methi (dried fenugreek), chili powder, and salt. Simmer until rich and creamy.
3. Add the cooked chicken to the sauce, simmer for 10–15 minutes, adjust seasoning, and serve hot with naan or rice.

Example 3:
Dish: Cheesy Bake
Summary: 1. Boil or steam your choice of veggies (e.g., broccoli, cauliflower, carrots, corn) or cooked pasta/chicken. Lightly season with salt, pepper, and herbs.
2. In a pan, melt butter, add flour, and cook for a minute. Slowly whisk in milk to make a smooth white sauce. Add grated cheese (like cheddar or mozzarella) and stir until melted and creamy.
3. Mix the base with the cheese sauce, pour into a greased baking dish, top with more cheese, and bake at 180°C (350°F) for 20–25 minutes until golden and bubbly. Serve hot!


Now generate the recipe."""

In [287]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": image_path,
            },
            {"type": "text", "text": recipe_generator},
        ],
    }
]

In [288]:
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
).to("cuda")

In [289]:
generated_ids = model.generate(
    **inputs,
    max_new_tokens=150,
    temperature = 0.3,
    # top_p=0.9,
    # top_k = 50,
    no_repeat_ngram_size=3,
    ).to("cpu")
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
recipe_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
generated_recipe = recipe_text[0]
print(recipe_text[0])

Dish Name: Penne Arrabdiata
Summary:
1. Cook penne noodles according to package instructions.
2\. In a large pot, combine crushed tomatoes, red wine, garlic powder, red pepper powder, salt, and black pepper. Bring to a boil, reduce heat, and simmer for about 12-15 mins.
3\. Add cooked penne to the tomato sauce and toss to coat. Serve garnished with parsley and Parmesan.


## ROUGE Score

In [290]:
reference_summary_path = f"./Test_Summaries/{filename}.txt"
with open(reference_summary_path, "r", encoding="utf-8") as ref_file:
    reference_summary = ref_file.read().strip()

In [291]:
scorer = rouge_scorer.RougeScorer(
    ['rouge1', 'rouge2', 'rougeL'], 
    use_stemmer=True
)

In [292]:
scores = scorer.score(reference_summary, generated_recipe)

In [293]:
print("ROUGE Scores:")
for metric, score in scores.items():
    print(f"{metric.upper()}:")
    print(f"  Precision: {score.precision:.4f}")
    print(f"  Recall:    {score.recall:.4f}")
    print(f"  F1:        {score.fmeasure:.4f}")
    print()

ROUGE Scores:
ROUGE1:
  Precision: 0.3810
  Recall:    0.4000
  F1:        0.3902

ROUGE2:
  Precision: 0.0484
  Recall:    0.0508
  F1:        0.0496

ROUGEL:
  Precision: 0.2540
  Recall:    0.2667
  F1:        0.2602

